In [2]:
%pylab inline
import numpy
import pandas

Populating the interactive namespace from numpy and matplotlib


In [53]:
data = pandas.read_csv('train.csv')

In [287]:
`

In [376]:
import pandas as pd

# read in the training and testing data into Pandas.DataFrame objects
input_df = pd.read_csv('train.csv', header=0)
submit_df  = pd.read_csv('test.csv',  header=0)

# merge the two DataFrames into one
cd = pd.concat([input_df, submit_df])

# re-number the combined data set so there aren't duplicate indexes
cd.reset_index(inplace=True)

# reset_index() generates a new column that we don't want, so let's get rid of it
cd.drop('index', axis=1, inplace=True)

# the remaining columns need to be reindexed so we can access the first column at '0' instead of '1'
cd = cd.reindex_axis(input_df.columns, axis=1)

print cd.shape[1], "columns:", cd.columns.values
print "Row count:", cd.shape[0]
cd

12 columns: ['PassengerId' 'Survived' 'Pclass' 'Name' 'Sex' 'Age' 'SibSp' 'Parch'
 'Ticket' 'Fare' 'Cabin' 'Embarked']
Row count: 1309


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


In [379]:
data = cd

In [381]:
# data = data.dropna()
age_mean = data['Age'].dropna().astype(int).mode()[0]
fare_mode = data['Fare'].dropna().astype(int).mode()[0]
em_mode = data['Embarked'].mode()[0]
print age_mean
print "fare", fare_mode
print em_mode

24
fare 7
S


In [214]:
ls = ["mr","mrs","miss","master"]
t = [0,0,0,0]
co = [0,0,0,0]
import math
def meanAge(da):
    st = transformName(da['Name'])
    if math.isnan(da['Age']):
        return
    for i in range(0,4):
        if ls[i] == st:
            t[i] = t[i] + float(da['Age'])
            co[i] = co[i] + 1
            break

# for dtd in data.iterrows():
#     meanAge(dtd[1])
data.apply(meanAge, axis=1)
print t,co
for i in range(0,4):
    t[i] = t[i]/co[i]
print t

[13912.5, 3949.0, 3179.0, 164.67000000000002] [422, 110, 146, 36]
[32.96800947867298, 35.9, 21.773972602739725, 4.574166666666667]


In [382]:
import re
rx = re.compile(u'[\W_]+', re.UNICODE)
ls = ["mr","mrs","miss","master"]
def transformName(st):                                
    if st is None:
        return ls[0]
    string = rx.sub(' ', st)
    for s in re.split(' ', string):
        if s.lower() in ls:
            return s.lower()
    return ls[0]

def transformAge(da):
    if math.isnan(da['Age']):
        st = transformName(da['Name'])
        for i in range(0,4):
            if ls[i] == st:
                da['Age'] = t[i]
                break
    return da
                  
def transform(x_data):
    temp = x_data;
    temp = pandas.concat([temp, pandas.get_dummies(temp['Name'].apply(transformName), prefix='Name')], axis=1)
    temp['Age'] = temp['Age'].fillna(age_mean)
#     temp = temp.apply(transformAge, axis = 1)
    temp['Embarked'] = temp['Embarked'].fillna(em_mode)
#     temp['Embarked'] = temp['Embarked'].fillna('N')
    temp['Sex'] = temp['Sex'].map({'female':0, 'male':1}).astype(int)
#     temp['Embarked'] = temp['Embarked'].map({'C':1, 'S':2, 'Q':3}).astype(int)
    temp = pandas.concat([temp, pandas.get_dummies(temp['Embarked'], prefix='Embarked')], axis=1)
    temp = temp.drop(['Embarked'],axis = 1)
    temp['Fare'] = temp['Fare'].fillna(fare_mode)
    
    # Replace missing values with "U0"
    temp['Cabin'][temp.Cabin.isnull()] = 'U0'

    # Create a feature for the deck
    temp['Deck'] = temp['Cabin'].map( lambda x : re.compile("([a-zA-Z]+)").search(x).group())
    temp['Deck'] = pandas.factorize(temp['Deck'])[0]

    # Create binary features for each deck
    decks = pandas.get_dummies(temp['Deck']).rename(columns=lambda x: 'Deck_' + str(x))
    temp = pandas.concat([temp, decks], axis=1)

    # Create feature for the room number
#     temp['Room'] = temp['Cabin'].map( lambda x : re.compile("([0-9]+)").search(x).group()).astype(int) + 1
    
    temp = temp.drop(['Ticket','Cabin','PassengerId','Name'],axis = 1)
    return temp


In [361]:
re.compile("([a-zA-Z]+)").search("").group()

AttributeError: 'NoneType' object has no attribute 'group'

In [383]:
tdata = transform(data)
train_data = tdata.values
print train_data
tdata.info()
tdata.head(5)

[[  0.   3.   1. ...,   0.   0.   0.]
 [  1.   1.   0. ...,   0.   0.   0.]
 [  1.   3.   0. ...,   0.   0.   0.]
 ..., 
 [ nan   3.   1. ...,   0.   0.   0.]
 [ nan   3.   1. ...,   0.   0.   0.]
 [ nan   3.   1. ...,   0.   0.   0.]]
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1309 entries, 0 to 1308
Data columns (total 24 columns):
Survived       891 non-null float64
Pclass         1309 non-null int64
Sex            1309 non-null int64
Age            1309 non-null float64
SibSp          1309 non-null int64
Parch          1309 non-null int64
Fare           1309 non-null float64
Name_master    1309 non-null float64
Name_miss      1309 non-null float64
Name_mr        1309 non-null float64
Name_mrs       1309 non-null float64
Embarked_C     1309 non-null float64
Embarked_Q     1309 non-null float64
Embarked_S     1309 non-null float64
Deck           1309 non-null int64
Deck_0         1309 non-null float64
Deck_1         1309 non-null float64
Deck_2         1309 non-null float64
De

/usr/local/lib/python2.7/dist-packages/IPython/kernel/__main__.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Name_master,Name_miss,Name_mr,...,Deck,Deck_0,Deck_1,Deck_2,Deck_3,Deck_4,Deck_5,Deck_6,Deck_7,Deck_8
0,0,3,1,22,1,0,7.2500,0,0,1,...,0,1,0,0,0,0,0,0,0,0
1,1,1,0,38,1,0,71.2833,0,0,0,...,1,0,1,0,0,0,0,0,0,0
2,1,3,0,26,0,0,7.9250,0,1,0,...,0,1,0,0,0,0,0,0,0,0
3,1,1,0,35,1,0,53.1000,0,0,0,...,1,0,1,0,0,0,0,0,0,0
4,0,3,1,35,0,0,8.0500,0,0,1,...,0,1,0,0,0,0,0,0,0,0


In [394]:
X = train_data[0:891,1:]
Y = train_data[0:891,0]
print train_data.shape
print X.shape, Y.shape
XTest = train_data[891:,1:]
print XTest.shape

(1309, 24)
(891, 23) (891,)
(418, 23)


In [408]:
# Instantiate and train the classifier
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors=1)

In [653]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators=100, verbose = 1,max_depth=4.0, max_features=0.20)
# classifier = RandomForestClassifier(n_estimators=1000, verbose = 1)

In [243]:
# DecisionTreeClassifier
from sklearn.tree import DecisionTreeClassifier
classifier = DecisionTreeClassifier(random_state=0)

In [230]:
classifier?

In [666]:
# split the data into training and validation sets
from sklearn.cross_validation import train_test_split
# from sklearn.decomposition import PCA
# pca = PCA(n_components=12)
# xs = pca.fit_transform(X)
xs = X
X_train, X_test, y_train, y_test = train_test_split(xs,Y)
# X_train = xs
# y_train = Y
# train the model
classifier.fit(X_train, y_train)

# use the model to predict the labels of the test data
predicted = classifier.predict(X_test)
expected = y_test
matches = (predicted == expected)
print matches.sum()
print len(matches)

190
223


[Parallel(n_jobs=1)]: Done   1 jobs       | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done   1 jobs       | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished


In [424]:
from sklearn.grid_search import GridSearchCV

parameter_grid = {
    'max_features': numpy.linspace(0.1,1.0,10),
    'max_depth': numpy.linspace(5,15,10)
}

grid_search = GridSearchCV(RandomForestClassifier(n_estimators = 100), parameter_grid,
                            cv=5, verbose=2)

In [425]:
grid_search.fit(X,Y)

Fitting 5 folds for each of 100 candidates, totalling 500 fits
[CV] max_features=0.1, max_depth=5.0 .................................
[CV] ........................ max_features=0.1, max_depth=5.0 -   0.1s
[CV] max_features=0.1, max_depth=5.0 .................................
[CV] ........................ max_features=0.1, max_depth=5.0 -   0.1s
[CV] max_features=0.1, max_depth=5.0 .................................
[CV] ........................ max_features=0.1, max_depth=5.0 -   0.1s
[CV] max_features=0.1, max_depth=5.0 .................................
[CV] ........................ max_features=0.1, max_depth=5.0 -   0.1s
[CV] max_features=0.1, max_depth=5.0 .................................
[CV] ........................ max_features=0.1, max_depth=5.0 -   0.1s
[CV] max_features=0.2, max_depth=5.0 .................................
[CV] ........................ max_features=0.2, max_depth=5.0 -   0.1s
[CV] max_features=0.2, max_depth=5.0 .................................
[CV] .........

[Parallel(n_jobs=1)]: Done   1 jobs       | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done  41 jobs       | elapsed:    5.2s



[CV] max_features=0.9, max_depth=5.0 .................................
[CV] ........................ max_features=0.9, max_depth=5.0 -   0.2s
[CV] max_features=0.9, max_depth=5.0 .................................
[CV] ........................ max_features=0.9, max_depth=5.0 -   0.2s
[CV] max_features=0.9, max_depth=5.0 .................................
[CV] ........................ max_features=0.9, max_depth=5.0 -   0.2s
[CV] max_features=0.9, max_depth=5.0 .................................
[CV] ........................ max_features=0.9, max_depth=5.0 -   0.1s
[CV] max_features=1.0, max_depth=5.0 .................................
[CV] ........................ max_features=1.0, max_depth=5.0 -   0.2s
[CV] max_features=1.0, max_depth=5.0 .................................
[CV] ........................ max_features=1.0, max_depth=5.0 -   0.2s
[CV] max_features=1.0, max_depth=5.0 .................................
[CV] ........................ max_features=1.0, max_depth=5.0 -   0.2s
[CV] 

[Parallel(n_jobs=1)]: Done 162 jobs       | elapsed:   22.0s
[Parallel(n_jobs=1)]: Done 365 jobs       | elapsed:   59.3s



[CV] max_features=0.4, max_depth=12.7777777778 .......................
[CV] .............. max_features=0.4, max_depth=12.7777777778 -   0.2s
[CV] max_features=0.4, max_depth=12.7777777778 .......................
[CV] .............. max_features=0.4, max_depth=12.7777777778 -   0.2s
[CV] max_features=0.4, max_depth=12.7777777778 .......................
[CV] .............. max_features=0.4, max_depth=12.7777777778 -   0.2s
[CV] max_features=0.4, max_depth=12.7777777778 .......................
[CV] .............. max_features=0.4, max_depth=12.7777777778 -   0.1s
[CV] max_features=0.4, max_depth=12.7777777778 .......................
[CV] .............. max_features=0.4, max_depth=12.7777777778 -   0.1s
[CV] max_features=0.5, max_depth=12.7777777778 .......................
[CV] .............. max_features=0.5, max_depth=12.7777777778 -   0.2s
[CV] max_features=0.5, max_depth=12.7777777778 .......................
[CV] .............. max_features=0.5, max_depth=12.7777777778 -   0.1s
[CV] 

[Parallel(n_jobs=1)]: Done 500 out of 500 | elapsed:  1.4min finished


GridSearchCV(cv=5,
       estimator=RandomForestClassifier(bootstrap=True, compute_importances=None,
            criterion='gini', max_depth=None, max_features='auto',
            max_leaf_nodes=None, min_density=None, min_samples_leaf=1,
            min_samples_split=2, n_estimators=100, n_jobs=1,
            oob_score=False, random_state=None, verbose=0),
       fit_params={}, iid=True, loss_func=None, n_jobs=1,
       param_grid={'max_features': array([ 0.1,  0.2,  0.3,  0.4,  0.5,  0.6,  0.7,  0.8,  0.9,  1. ]), 'max_depth': array([  5.     ,   6.11111,   7.22222,   8.33333,   9.44444,  10.55556,
        11.66667,  12.77778,  13.88889,  15.     ])},
       pre_dispatch='2*n_jobs', refit=True, score_func=None, scoring=None,
       verbose=2)

In [426]:
sorted(grid_search.grid_scores_, key=lambda x: x.mean_validation_score)
grid_search.best_score_
grid_search.best_params_

{'max_depth': 9.4444444444444446, 'max_features': 0.80000000000000004}

In [78]:
test.head(10)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
5,897,3,"Svensson, Mr. Johan Cervin",male,14.0,0,0,7538,9.2250,NaN,S
6,898,3,"Connolly, Miss. Kate",female,30.0,0,0,330972,7.6292,NaN,Q
7,899,2,"Caldwell, Mr. Albert Francis",male,26.0,1,1,248738,29.0000,NaN,S
8,900,3,"Abrahim, Mrs. Joseph (Sophie Halaut Easu)",female,18.0,0,0,2657,7.2292,NaN,C
9,901,3,"Davies, Mr. John Samuel",male,21.0,2,0,A/4 48871,24.1500,NaN,S


In [218]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 418 entries, 0 to 417
Data columns (total 11 columns):
PassengerId    418 non-null int64
Pclass         418 non-null int64
Name           418 non-null object
Sex            418 non-null object
Age            332 non-null float64
SibSp          418 non-null int64
Parch          418 non-null int64
Ticket         418 non-null object
Fare           417 non-null float64
Cabin          91 non-null object
Embarked       418 non-null object
dtypes: float64(2), int64(4), object(5)
memory usage: 39.2+ KB


In [353]:
def writeFile(y_data):
    f = open('oplat','w')
    f.write("PassengerId,Survived\n")
    i = 892;
    for k in y_data:
        f.write(str(i) + "," + str(int(k)) + "\n")
        i = i+1
    f.close()

In [667]:
resul = classifier.predict(XTest)
print resul
writeFile(resul)

[ 0.  0.  0.  0.  1.  0.  1.  0.  1.  0.  0.  0.  1.  0.  1.  1.  0.  0.
  1.  1.  0.  0.  1.  0.  1.  0.  1.  0.  0.  0.  0.  0.  1.  1.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  1.  1.  0.  0.  0.  1.  1.  0.  0.  1.  1.
  0.  0.  0.  0.  0.  1.  0.  0.  0.  1.  1.  1.  1.  0.  0.  1.  1.  0.
  0.  0.  1.  0.  0.  1.  0.  1.  1.  0.  0.  0.  0.  0.  1.  0.  0.  1.
  1.  0.  1.  0.  0.  0.  1.  0.  0.  0.  1.  0.  0.  0.  1.  0.  0.  0.
  0.  0.  0.  1.  1.  1.  1.  0.  0.  1.  0.  1.  1.  0.  1.  0.  0.  1.
  0.  1.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.
  0.  0.  1.  0.  0.  0.  1.  0.  0.  1.  0.  0.  1.  0.  0.  1.  1.  1.
  1.  0.  0.  1.  0.  0.  1.  0.  0.  0.  0.  0.  0.  1.  1.  0.  1.  1.
  0.  1.  1.  0.  1.  0.  1.  0.  0.  0.  0.  0.  0.  0.  1.  0.  1.  1.
  0.  1.  1.  1.  0.  1.  0.  0.  1.  0.  1.  0.  0.  0.  0.  1.  0.  0.
  1.  0.  1.  0.  1.  0.  1.  0.  1.  1.  0.  1.  0.  0.  0.  1.  0.  0.
  0.  0.  0.  0.  1.  1.  1.  1.  0.  0.  0.  0.  1

[Parallel(n_jobs=1)]: Done   1 jobs       | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished


In [286]:
data['Age'].dropna().describe()

count    714.000000
mean      29.699118
std       14.526497
min        0.420000
25%       20.125000
50%       28.000000
75%       38.000000
max       80.000000
Name: Age, dtype: float64